In [132]:
import pygame as pg
import numpy as np
import pdb


[<Surface(79x102x32 SW)>, <Surface(79x103x32 SW)>, <Surface(79x103x32 SW)>]

In [128]:
class Tiles:
    def __init__(self, tile_count):
        self.number = np.zeros((tile_count,),dtype = int)
        self.grid_pos = np.zeros((tile_count,))
        self.icon =[]
    
        gird_pos = np.random.permutation(16)
        for index in range(tile_count):
            self.number[index] = np.random.randint(1,4)
            self.icon.append(pg.image.load('C:/Henry/PythonProjects/Pygame/Threes/images/_' + str(self.number[index]) + '.png'))
            self.grid_pos[index] = gird_pos[index]
            self.drawTile(index)
        
        
        
    def determineMove(self,keys):
        current_move = 0

        if keys[pg.K_LEFT]:
            move_order = np.argsort(self.grid_pos)
            current_move = -4
        elif keys[pg.K_UP]:
            move_order = np.argsort(self.grid_pos)
            current_move = -1
    
        elif keys[pg.K_RIGHT]:
            move_order = np.argsort(self.grid_pos)
            move_order = move_order[-1:0:-1]
            current_move = 4
        elif keys[pg.K_DOWN]:
            move_order = np.argsort(self.grid_pos)
            move_order = move_order[-1:0:-1]
            current_move = 1
        else:
            move_order = np.argsort(self.grid_pos)
        return current_move, move_order
    def updatePosition(self,current_move, move_order):
        for index  in range(len(self.number)):
            if current_move != 0:
                print('current_move = ' + str(current_move))
                if self.grid_pos[index]+ current_move >=0 and self.grid_pos[index]+ current_move <=16:
                    if sum(np.isin(self.grid_pos,self.grid_pos[index]+current_move))==0:
                        self.grid_pos[index]+=current_move
                    else:
                        blockIndex = np.where(self.grid_pos==self.grid_pos[index]-1)
                        blockVal = self.number[blockIndex]
                        if self.number[index] == 1 or self.number[index] == 2:
                            if blockVal == 1 or blockVal ==2:
                                print('combine to make three')
                            else:
                                print('move blocked!')

                        elif self.number[index] == blockVal:
                            print('combine values and remove one tile')
                        else:
                            if current_move !=0:
                                print('move blocked')
            self.drawTile(index)
        return self
    def drawTile(self,index):
        current_y = np.mod(self.grid_pos[index],4)
        current_x = np.floor(self.grid_pos[index]/4)
        screen.blit(self.icon[index],(current_x*self.icon[index].get_width(),current_y*self.icon[index].get_height()))
        
        
        
        
        
        
  
        
        return self
   
    

In [133]:
running = True
current_x = 0
current_y = 0
screen = pg.display.set_mode((800,600))
screen.fill((0,0,255))
pg.display.set_caption("Threes!")


grid  =pg.image.load('C:/Henry/PythonProjects/Pygame/Threes/images/grid.png')
tile_list = [1,2,3,6,12,24,48,96,192,384,768,1536,3072,6144]
start_num = 2

game_tiles = Tiles(3)
game_tiles.icon


obstacles_pos = []
ex_state=True

while running:
    #pg.time.delay(100)
    screen.fill((0,0,255))
    screen.blit(grid,(0,0))
    keys = pg.key.get_pressed()
#     pdb.set_trace()
    if ex_state and sum(keys)>0:
        current_move, move_order = game_tiles.determineMove(keys)
        
        ex_state = False
    else:
        current_move = 0
        move_order = np.argsort(game_tiles.grid_pos)
    game_tiles = game_tiles.updatePosition(current_move, move_order)
        
    
            
    pg.display.update()

    
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
            pg.display.quit()
        if event.type == pg.MOUSEBUTTONDOWN:
            print('mouse clicked')
        if event.type == pg.KEYUP:
            ex_state = True;

    
      
            

> <ipython-input-133-b7257ffe6028>(26)<module>()
-> if ex_state:
(Pdb) keys
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

BdbQuit: 

In [134]:
pg.display.quit()

In [118]:
current_move

array([0, 0])